TODO: Fix this to run the flask app from here.

In [ ]:
#!pip install flask pyopenssl

#import subprocess

#process = subprocess.Popen('flask --app webserver run', stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

#!flask --app webserver run

In [ ]:
import requests

def send_prediction_request(string_to_predict):
    return requests.post('http://127.0.0.1:5000/', json={'comment': string_to_predict}).text


print(send_prediction_request('test comment 1'))
print(send_prediction_request('test comment 2'))